In [1]:
from bcb import sgs
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import sidrapy as sidra
from functools import reduce
# Definindo o estilo do matplotlib para os gráficos
mpl.style.use('bmh')

# Dicionário com as séries desejadas
series = {
    'BCB: PIB Mensal': 4380,
    'BCB: Papel Moeda em Circulação': 1786,
    'BCB: Reservas Bancárias': 1787,
    'BCB: Papel Moeda em Poder do Público': 27789,
    'BCB: M1': 27791,
    'BCB:M2': 27810,
    'BCB:M3': 27813,
    'BCB:M4': 27815
}

# Baixando os dados do Banco Central usando o `bcb` para todas as séries
bcb_data = sgs.get(series, start='2002-02-01')
bcb_data.index = bcb_data.index.to_period('M')

# Exibindo as primeiras linhas do DataFrame
print(bcb_data.head())

         BCB: PIB Mensal  BCB: Papel Moeda em Circulação  \
Date                                                       
2002-02         111477.1                        32102817   
2002-03         118444.7                        32547930   
2002-04         120385.9                        31178230   
2002-05         123552.5                        32930327   
2002-06         123424.4                        33899629   

         BCB: Reservas Bancárias  BCB: Papel Moeda em Poder do Público  \
Date                                                                     
2002-02                 15788389                              26732167   
2002-03                 16306755                              27191413   
2002-04                 15567366                              26006602   
2002-05                 18582861                              26996297   
2002-06                 16792917                              28281540   

          BCB: M1     BCB:M2     BCB:M3     BCB:M4  
Date   

In [14]:
def get_sidra_series(column_name: str, table_code:str, variable:str, classifications:dict) -> pd.DataFrame:
    
    """
    column_name: str
        Nome da coluna que será atribuída ao DataFrame final.
    table_code: str
        Código da tabela do SIDRA.
    variable: str
        Código da variável que será extraída.
    classifications: str
        Código da classificação que será extraída.
    """

    if classifications:
        df_sidra_brute = sidra.get_table(
            table_code=table_code, 
            territorial_level="1", 
            ibge_territorial_code="all", 
            period="all", 
            variable=variable, 
            classifications=classifications
        )
    else:
        df_sidra_brute = sidra.get_table(
            table_code=table_code, 
            territorial_level="1", 
            ibge_territorial_code="all", 
            period="all", 
            variable=variable
        )
    # Remover as linhas que não contêm dados relevantes (supondo que sejam as duas primeiras linhas)
    df_sidra = df_sidra_brute.iloc[2:].reset_index(drop=True)
    
    # Renomear as colunas abreviadas para nomes mais descritivos
    df_sidra.rename(columns={
        'D2C': 'Mês (Código)',
        'V': 'Valor'
    }, inplace=True)
    
    # Selecionar apenas as colunas necessárias
    df_sidra = df_sidra[['Mês (Código)', 'Valor']]
    
    # Converter 'Mês (Código)' em formato datetime
    df_sidra['Date'] = pd.to_datetime(df_sidra['Mês (Código)'], format='%Y%m')

    # Remover a coluna 'Mês (Código)', pois não é mais necessária
    df_sidra.drop(columns=['Mês (Código)'], inplace=True)
    
    # Definir 'Date' como índice
    df_sidra.set_index('Date', inplace=True)

    df_sidra.index = df_sidra.index.to_period('M')
    
    # Converter a coluna 'Valor' para numérico
    df_sidra['Valor'] = pd.to_numeric(df_sidra['Valor'], errors='coerce')
    
    # Renomear a coluna 'Valor' para o nome desejado
    df_sidra.rename(columns={'Valor': column_name}, inplace=True)

    if isinstance(df_sidra.columns, pd.MultiIndex):
        df_sidra.columns = [' '.join(col).strip() for col in df_sidra.columns.values]
    
    return df_sidra

In [ ]:
sidra_list = [
    {
        'column_name': 'IBGE: Indicador de Produção - Indústria de Transformação',
        'table_code': '8888',
        'variable': '12606',
        'classifications': {'544': '129316'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Indústria Extrativa',
        'table_code': '8888',
        'variable': '12606',
        'classifications': {'544': '129315'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens de Capital',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129278'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens Intermediários',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129283'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens de Consumo Duráveis',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129301'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Bens de Consumo Semi-Duráveis',
        'table_code': '8887',
        'variable': '12606',
        'classifications': {'543': '129306'}
    },
    {
        'column_name': 'IBGE: Índice de Receita Nominal de Vendas no Comércio Varejista',
        'table_code': '8880',
        'variable': '7169',
        'classifications': {'11046': '56733'}
    },
    {
        'column_name': 'IBGE: Indicador de Produção - Insumos Típicos da Construção Civil',
        'table_code': '8889',
        'variable': '12606',
        'classifications': {'25': '214'}
    },
    {
        'column_name': 'IBGE: Índice Nacional de Preços ao Consumidor Amplo (IPCA)',
        'table_code': '1737',
        'variable': '2266',
        'classifications': None
    }
]  

In [ ]:
def get_sidra_data(sidra_list: list) -> pd.DataFrame:
    
    """
    sidra_list: list
        Lista de dicionários com as informações necessárias para baixar os dados do SIDRA.
    """
    
    # Inicializar um DataFrame vazio
    df_list = []
    
    for item in sidra_list:
        column_name = item['column_name']
        table_code = item['table_code']
        variable = item['variable']
        classifications = item['classifications']
        
        df_item = get_sidra_series(column_name, table_code, variable, classifications)
        
        df_list.append(df_item)

        df_merged = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), df_list)

    # Interessado somente nos dados a partir do ano de 2003
    df_merged = df_merged[df_merged.index >= '2003-01-01']
    
    return df_merged

In [17]:
df_sidra = get_sidra_data(sidra_list)
df_sidra

,IBGE: Indicador de Produção - Indústria de Transformação,IBGE: Indicador de Produção - Indústria Extrativa,IBGE: Indicador de Produção - Bens de Capital,IBGE: Indicador de Produção - Bens Intermediários,IBGE: Indicador de Produção - Bens de Consumo Duráveis,IBGE: Indicador de Produção - Bens de Consumo Semi-Duráveis,IBGE: Índice de Receita Nominal de Vendas no Comércio Varejista,IBGE: Índice de Produção - Insumos Típicos da Construção Civil,IBGE: Índice Nacional de Preços ao Consumidor Amplo (IPCA)
Date,,,,,,,,,
1980-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.122300e-09
1980-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.497300e-09
1980-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.010400e-09
1980-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.486700e-09
1980-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.002770e-08
...,...,...,...,...,...,...,...,...,...
2024-06,104.26926,110.60854,99.21515,104.33944,106.16956,91.54800,109.37892,102.66006,6.941510e+03
2024-07,111.81249,112.92576,101.84117,110.80010,123.90949,104.49181,113.13572,112.64940,6.967890e+03
2024-08,113.34729,116.53595,103.29727,111.78391,132.56065,109.14190,114.22835,113.07725,6.966500e+03


In [ ]:
bcb_data.join(df_sidra, how='left')

AttributeError: 'builtin_function_or_method' object has no attribute 'is_unique'